# Using Langchain


## Start with local collection

In [15]:
import os
import openai
from pinecone import Pinecone
import langchain
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as pine
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

from langchain_experimental.smart_llm import SmartLLMChain
from langchain.prompts import PromptTemplate

In [2]:
os.environ['OPENAI_API_KEY'] = "openai-key"

In [33]:
openai_api_key = "openai-key"

#### Loading Basic utilities

In [3]:
def load_docs(directory_path):
    loader = DirectoryLoader(directory_path)
    documents = loader.load()
    return documents

In [4]:
def split_docs(documents, chunk_size=500, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                   chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

In [23]:
def create_backend(build_index=False, docs=None, chain_type="stuff", revise=True):

    # OpenAI credentials
    model_name = "gpt-3.5-turbo"

    # Pinecone credentials
    pinecone_api_key = "pinecone-key"
    environment = "gcp-starter"
    index_name = "rag-news-project"

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    if not revise:
        llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
        chain = load_qa_chain(llm, chain_type=chain_type)
    else:
        template = """\
            Context information is below.
            ---------------------
            {context}
            ---------------------
            Given the context information and not prior knowledge, answer the query.
            Query: {query}
            Answer: \
            """
        prompt = PromptTemplate.from_template(template)
        llm = ChatOpenAI(temperature=0, model_name="gpt-4")
        chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)
        
        pc = Pinecone(api_key=pinecone_api_key)
    
    if build_index and docs is not None:
        vector_store = Pinecone.from_documents(docs, embeddings, index_name=index_name)
    else:
        index = pc.Index(index_name)
        vector_store = pine(index, embeddings, "text")
    return chain, embeddings, vector_store

### Similarity and Lang

In [24]:
def get_similiar_docs(index, query, k=2, score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query, k=k)
    else:
        similar_docs = index.similarity_search(query, k=k)
    return similar_docs

def get_answer(chain, index, query):
    similar_docs = get_similiar_docs(index, query)
    answer = chain.run(input_documents=similar_docs, question=query)
    return answer

In [25]:
def get_answer_revise(chain, index, query):
    similar_docs = get_similiar_docs(index, query)
    context = "\n\n".join([doc.page_content for doc in similar_docs])
    answer = chain.run({'context':context, 'query':query})
    return answer

In [26]:
chain, embeddings, vector_store = create_backend(build_index=False, docs=None, revise=True)

In [27]:
### Question and Answer

In [28]:
qry = "Why is Supreme Court warning Patanjali?"

In [29]:
get_similiar_docs(vector_store, qry)

[Document(page_content='"The Supreme Court has issued a stern warning to Patanjali Ayurveda, cautioning against misleading advertisements and false claims targeting modern medicines and vaccinations.\\nIn response to a petition filed by the Indian Medical Association (IMA), the court has instructed Patanjali to refrain from making any deceptive statements and has threatened significant penalties for non-compliance.\\nA Bench of Justices Ahsanuddin Amanullah and Prashant Kumar Mishra emphasised that the issue could not be', metadata={'Author': 'Moneycontrol News', 'Date': 'November 21, 2023', 'Link': 'https://www.moneycontrol.com/news/business/markets/supreme-court-warns-patanjali-ayurveda-over-deceptive-medical-claims-threatens-severe-penalties-11783361.html', 'Title': 'A Bench of Justices Ahsanuddin Amanullah and Prashant Kumar Mishra stressed that the issue could not be reduced to a debate between allopathy/modern medicine and Ayurvedic products.'}),
 Document(page_content="Supreme C

In [30]:
get_answer_revise(chain, vector_store, qry)

/usr/local/google/home/dpksharma/PersonalProjects/Workshops/rag_workshop/rag_workshop/ragws/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SmartLLMChain chain...
Prompt after formatting:
            Context information is below.
            ---------------------
            "The Supreme Court has issued a stern warning to Patanjali Ayurveda, cautioning against misleading advertisements and false claims targeting modern medicines and vaccinations.\nIn response to a petition filed by the Indian Medical Association (IMA), the court has instructed Patanjali to refrain from making any deceptive statements and has threatened significant penalties for non-compliance.\nA Bench of Justices Ahsanuddin Amanullah and Prashant Kumar Mishra emphasised that the issue could not be

Supreme Court has urged the Union to present a proposal aimed at tackling misleading medical advertisements more broadly. The move comes in response to concerns raised by the IMA regarding Patanjali's campaigns that seemingly criticized modern medicine and vaccination efforts.\nThe Supreme Court's intervention underscores the gravity of the is

"The Supreme Court has issued a warning to Patanjali Ayurveda due to their misleading advertisements and false claims that target modern medicines and vaccinations. This action was taken in response to a petition filed by the Indian Medical Association (IMA), which expressed concerns about Patanjali's campaigns that seemingly criticized modern medicine and vaccination efforts. The court has instructed Patanjali to refrain from making any deceptive statements and has threatened significant penalties for non-compliance. Furthermore, the Supreme Court has urged the Union to present a proposal aimed at tackling misleading medical advertisements more broadly, highlighting the seriousness of the issue and the need for responsible advertising in the medical field."

In [34]:
llm = ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=0)